## **PFD Query using langchain and AstraDB**

In [2]:
!pip install -q cassio datasets openai tiktoken

Importing all the necessary packages


In [3]:
!pip install langchain-community

In [4]:
!pip install google-generativeai

In [5]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
!pip install PyPDF2

In [7]:
from PyPDF2 import PdfReader

In [8]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:pxYZmJNJJgaSnsPDjhCALtEI (Replace with your Token))"
ASTRA_DB_ID = "2964c6ae-6a4b (Replace with your Astra DB id))"


In [9]:
import google.generativeai as GenAI
GEMINI_API_KEY = "API-KEY"

In [10]:
pdf_reader = PdfReader('/content/Question bank_TSA.pdf')

In [11]:
raw_text = ''
for i, page in enumerate(pdf_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text
raw_text[555:1000]

'  \nText Preprocessing and Wrangling – Text tokenization – Stemming – Lemmatization – Removing stop - \nwords – Feature Engineering for Text representation – Bag of Words model - Bag of N -Grams model – \nTF-IDF model – Case study: Analysis of Text Preprocessing using NLTK, Implementation of TF -IDF \nmodels.  \n \nPart – A \n1 State text tokenization in text preprocessing?  \n\uf0b7 Tokenization  is the process of dividing text into a set of meaningful '

Initializing connection to Data Base

In [12]:
import cassio
cassio.init(token = ASTRA_DB_APPLICATION_TOKEN, database_id = ASTRA_DB_ID)

In [ ]:
!pip install vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.79.0
    Uninstalling google-cloud-aiplatform-1.79.0:
      Successfully uninstalled google-cloud-aiplatform-1.79.0


In [33]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00


In [41]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model = "gemini-2.0-pro-exp-02-05", temperature=0)
embedding = HuggingFaceEmbeddings()

<ipython-input-41-8ad708ef22ba>:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


LangChain Vector store (Backend - AstraDB)

In [17]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name = 'pdf_query',
    session  = None,
    keyspace= None
)

In [18]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len
    )
texts = text_splitter.split_text(raw_text)

In [21]:
texts[:50]

['National Engineering College, K.R.Nagar, Kovilpatti -628503  \n(An Autonomous Institution, Affiliated to Anna University, Chennai)  \nDepartment of Artificial Intelligence & Data Science  \nQuestion Bank  \n                                                                         \nCourse Code & Title  : 19AD08E & TEXT AND SPEECH ANALYTICS  \nProgram  : B. Tech (AI & DS)  \nSemester  : VI \nRegulation  : 2019  \nCredit  : 3 \nCourse Instructors  : Ms. P. Swarna Gowsalya, AP/AI & DS  \nCourse Coordinator  :   Ms.V.Veera Anusuya, AP/AI & DS  \n \nCO1 - Introduction  \nText Preprocessing and Wrangling – Text tokenization – Stemming – Lemmatization – Removing stop - \nwords – Feature Engineering for Text representation – Bag of Words model - Bag of N -Grams model – \nTF-IDF model – Case study: Analysis of Text Preprocessing using NLTK, Implementation of TF -IDF \nmodels.  \n \nPart – A \n1 State text tokenization in text preprocessing?',
 'models.  \n \nPart – A \n1 State text tokenizatio

Load the Data into the Vector Store

In [23]:
astra_vector_store.add_texts(texts[:50])
print("%i number of Data loaded into the Vector Store" % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

22 number of Data loaded into the Vector Store


In [43]:

first_question = True
while True:
  if first_question:
    question = input("Enter your question or type 'quit' to exit : ").strip()
  else:
    question = input("\nDo you have any other queries (or type 'quit' to exit): ").strip()

  if question.lower() == "quit":
    break
  if question == "":
    continue

  first_question = False

  response = astra_vector_index.query(question, llm = llm)
  print(response)

  for answer, score in astra_vector_store.similarity_search_with_score(question, k = 4):
    print("    [%0.4f] \"%s ...\"" % (score, answer.page_content[:84]))


Enter your question or type 'quit' to exit : TF-IDF


Here's a breakdown of how to calculate the TF-IDF scores for the terms "data" and "future" based on the provided context:

**Understanding TF-IDF**

*   **TF (Term Frequency):** Measures how often a term appears in a document.
    *   TF = (Number of times the term appears in the document) / (Total number of terms in the document)

*   **IDF (Inverse Document Frequency):** Measures how rare a term is across all documents.
    *   IDF = log(Total number of documents / Number of documents containing the term)

*   **TF-IDF:** The product of TF and IDF. It gives a high score to terms that are frequent in a specific document but rare across the entire corpus.
    *   TF-IDF = TF \* IDF

**Calculations**
**1. Term Frequency (TF):**
*   **"Data":**
    *   Sentence 1: 1/7
    *   Sentence 2: 1/7
    *   Sentence 3: 0/4
*   **"Future":**
    *   Sentence 1: 1/7
    *   Sentence 2: 0/7
    *   Sentence 3: 1/4

**2. Inverse Document Frequency (IDF):**
*    Total number of documents:3
*   **"Dat

I don't have enough information to answer the question. The provided context defines feature engineering, stop-word removal, Bag of Words, and discusses question-answering systems, but it doesn't define the term "semantic".

    [0.6599] "example of a QA system based on information retrieval, detailing its architecture 
a ..."
    [0.6599] "example of a QA system based on information retrieval, detailing its architecture 
a ..."
    [0.6571] "the dataset.  
 Enhances Computational Efficiency:  Eliminating stop -words reduces ..."
    [0.6571] "the dataset.  
 Enhances Computational Efficiency:  Eliminating stop -words reduces ..."

Do you have any other queries (or type 'quit' to exit): quit
